Initiliase les lignes et regarde les titres les plus récurrents

In [ ]:
import matplotlib.pyplot as plt

with open("../fichier_sans_lignes_vides.txt", "r", encoding="utf8") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines if line.strip()]

d_title = {}
for i in range(len(lines)):
    if lines[i][0:5] == "<text":
        p = ""
        boo = False
        for j in range(len(lines[i])):
            if lines[i][j-7:j] == 'title="':
                boo = True
            if lines[i][j] == '"':
                boo = False
            if boo != False:
                p += lines[i][j]
        if p in d_title:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
            d_title[p].append(i)
        else:
            d_title[p] = [i]
    if i > 50:
        break 
# print(d_title)

# for elt in d_title:
#     if len(d_title[elt]) > 1:
#         print(elt,"---",len(d_title[elt]))  
#affiche le nombre de thèmes qui apparaisse + de 2 fois, et c'est souvent 2 fois max donc pas fou de faire une analyse sur les thèmes sauf si on les généralise mais c'est risqué sur des questions précises
# pas concluant

Regarde les questions et les mets dans un dictionnaire en fonction d'un département

In [ ]:
# d_dept_quest = {}
# for i in range(len(lines)):
#     if lines[i][0:5] == "<text":
#         p = ""
#         boo = False
#         for j in range(len(lines[i])):
#             # if lines[i][j-12:j] == 'department="': #par nom de departement
#             # if lines[i][j-8:j] == 'region="':  #par region
#             if lines[i][j-15:j] == 'departmentiso="':  #par numéro de departement
#                 boo = True
#             if lines[i][j] == '"':
#                 boo = False
#             if boo != False:
#                 p += lines[i][j]
#         if p in d_dept_quest:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
#             # print(lines[i])
#             d_dept_quest[p].append(lines[i])
#         else:
#             d_dept_quest[p] = [lines[i]]
    # if i > 100:
    #     break

   # for elt in d_dept_quest:
#     d_dept_quest[elt] = len(d_dept_quest[elt])
#     print(elt,"---",d_dept_quest[elt]) 
# print("\n\n")

La même chose en faisant les réponses

In [ ]:
d_dept_rep = {}
for i in range(len(lines)):
    if lines[i][0:5] == "<text":
        p = ""
        boo = False
        for j in range(len(lines[i])):
            # if lines[i][j-12:j] == 'department="': #par nom de departement
            # if lines[i][j-8:j] == 'region="':  #par region
            if lines[i][j-15:j] == 'departmentiso="':  #par numéro de departement
                boo = True
            if lines[i][j] == '"':
                boo = False
            if boo != False:
                p += lines[i][j]
    else:
        if lines[i] != "</text>":
            if p in d_dept_rep:  #ajoute dans un dictionnaire le thème et son itération correspondante dans une liste
                # print(lines[i])
                d_dept_rep[p].append(lines[i])
            else:
                d_dept_rep[p] = [lines[i]]
    # if i > 10000:
    #     break

list_dep = ["FR-09","FR-11","FR-12","FR-30","FR-31","FR-32","FR-34","FR-46","FR-48","FR-65","FR-66","FR-81","FR-82"]
for cle in list(d_dept_rep.keys()):
    if cle not in list_dep:
        del d_dept_rep[cle]
# for elt in d_dept_rep:
#     d_dept_rep[elt] = len(d_dept_rep[elt])
#     print(elt,"---",d_dept_rep[elt]) 
#par départment c'est déjà mieux (données sur 100 départements)

#trier de manière décroissante
sorted_d = dict(sorted(d_dept_rep.items(), key=lambda x: len(x[1]), reverse=True))

In [ ]:
def rep_dep(num_dep):
    return sorted_d["FR-"+str(num_dep)]

# print(rep_dep(34))

- Par thèmes: on prend le fichier contenant toutes les réponses pour chaque département et on regarde le thème associé à ces réponses et on fait ça pour chaque département. Et ensuite on compare avec les 10 mots les plus utilisés pour chaque département et on regarde si c'est similaire.

- Par dépendance: on prend le fichier contenant les réponses possédant au moins un des 10 mots les plus utilsés dans le département. Puis on prend un mot clé en particulier (ex:impôt) et on regarde tous les mots qui dépendent de celui-ci. On fait ça pour toutes les réponses du fichier et on crée un nuage de mots pour chaque département avec au centre le mot clé et autour les mots qui en dépendent où leur grosseur varie en fonction du nombre de fois qu'il apparaît 

In [16]:
import spacy
from spacy import displacy

nlp = spacy.load('fr_core_news_md')


with open("Dep avec phrase filtre.txt", "r", encoding="utf8") as f:
    lines = f.readlines()

# Créer un dictionnaire vide
occitanie_dico = {}


# print(len(lines)) #13 departemeent et saute une ligne
# Parcourir chaque ligne du fichier
for line in lines:
  if line[0:3]=="FR-":
    key = line[0:5]
    occitanie_dico[key] = line[6:]

print(occitanie_dico["FR-11"])


# # Phrase à analyser
# doc = nlp("Reviens ici")

# # Boucle pour afficher les informations de chaque token
# for token in doc:
#     print(token.text, token.pos_, token.dep_)


 Ne pas attendre après des bénévoles pour nourrir la population', 'Les gouvernants doivent rendre des comptes sur leurs dépenses. ', 'Aucune', 'Baisser tous les impôts surtout pour les classes moyennes qui sont les grands oubliés comme d habitude. ', 'Impôts sur le revenu, impôts fonciers. Augmenter les remboursements de la sécurité sociale', 'Aucun', 'Non', Abolir dès a présent la taxe d'habitation et réduire les impôts sur le revenu pour les classes moyennes. , 'Impôt sur le revenu. marre de passer ma vie a payer , entre les taxes ,les impôts,toutes les factures,les loyers etc', 'Des aides sociales pour les classes moyennes. ', 'Aucunes', 'Augmenter le salaire des aides soignants , métier dont la pénibilité est énorme, qui sont payés comme des miséreux alors qu ils sont en sous effectifs en permanence. Baisser les impôts sur le revenu et impôts fonciers. ', 'Impôt sur le revenu. ', 'RIEN', 'Aucun', 'Augmenter le salaire des aides soignants qui est une profession oubliée et payée comm

In [18]:
l = ["ddd","frfr"]
print(", ".join(l))
print(type(", ".join(l)))

ddd, frfr
<class 'str'>
